In [1]:
import pandas

import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

Definisanje globalnih promenljivih za lakse menjanje

In [2]:
PUTANJA = 'C:/Users/User/Desktop/AI'
KATEGORIJE = ['ZDRAV', 'VIRUS', 'BAKTERIJA', 'STREPTOCOCCUS', 'COVID-19', 'ARDS', 'SARS']
IMG_SIZE = 250

Pravljenje niza labela "train_y" i punimo ga brojevnim reprezentacijama zadatih kategorija

In [3]:
csv_fajl = pandas.read_csv(PUTANJA + r"\chest_xray_metadata.csv")  # promeniti na svoju lokaciju

# uklanjamo pusace zbog manjka slika (takodje je tako navedeno u zadatku)

train_y = np.ndarray(0)

for index, row in csv_fajl.iterrows():
    if row['Label'] != 'Normal':
        if str(row['Label_2_Virus_category']) != 'nan':
            if row['Label_2_Virus_category'] == 'Streptococcus':
                #csv_fajl.at[str(index), 'Label'] = 3
                train_y = np.append(train_y, 3)
                #row['Label'] = 3
            elif row['Label_2_Virus_category'] == 'COVID-19':
                #csv_fajl.at[str(index), 'Label'] = 4
                train_y = np.append(train_y, 4)
                #row['Label'] = 4
            elif row['Label_2_Virus_category'] == 'ARDS':
                #csv_fajl.at[str(index), 'Label'] = 5
                train_y = np.append(train_y, 5)
                #row['Label'] = 5
            elif row['Label_2_Virus_category'] == 'SARS':
                #csv_fajl.at[str(index), 'Label'] = 6
                train_y = np.append(train_y, 6)
                #row['Label'] = 6
            else:
                print('promaseno ' + row['Label_2_Virus_category'])
            # print(row['Label'], " => " , row['Label_1_Virus_category'], "|", row['Label_2_Virus_category'])#, row['X_ray_image_name'])
        else:
            if row['Label_1_Virus_category'] == 'Virus':
                #csv_fajl.at[str(index), 'Label'] = 1
                train_y = np.append(train_y, 1)
                #row['Label'] = 1
            elif row['Label_1_Virus_category'] == 'bacteria':
                #csv_fajl.at[str(index), 'Label'] = 2
                train_y = np.append(train_y, 2)
                #row['Label'] = 2
            else:
                print('promaseno ' + row['Label_1_Virus_category'])
            # print(row['Label'], " => " , row['Label_1_Virus_category'], "|", row['Label_2_Virus_category'])#, row['X_ray_image_name'])
    else:
        #csv_fajl.at[str(index), 'Label'] = 0
        train_y = np.append(train_y, 0)
        #row['Label'] = 0

In [4]:
csv_fajl.head()
training_data = []

for index, row in csv_fajl.iterrows():
    img_name = row['X_ray_image_name']
    lokacija = PUTANJA + '/snimci/' + str(img_name)
    try:# za slucaj da neka slika ostecena 
        slika = cv2.imread(lokacija, cv2.IMREAD_GRAYSCALE)
        slika = cv2.resize(slika, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
        training_data.append([slika, int(train_y[index])])
    except:
        print("Ostecena slika " + str(img_name))

In [5]:
print(len(training_data))
"""
for index, row in csv_fajl.iterrows():
    img_name = row['X_ray_image_name']
    lokacija = PUTANJA + '/snimci/' + str(img_name)
    try:# za slucaj da neka slika ostecena 
        slika = cv2.imread(lokacija, cv2.IMREAD_GRAYSCALE)
        slika = cv2.resize(slika, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
        print(str(train_y[index]) + " ==> " + str(img_name))
    except:
        print("Ostecena slika " + str(img_name))
"""

5286


'\nfor index, row in csv_fajl.iterrows():\n    img_name = row[\'X_ray_image_name\']\n    lokacija = PUTANJA + \'/snimci/\' + str(img_name)\n    try:# za slucaj da neka slika ostecena \n        slika = cv2.imread(lokacija, cv2.IMREAD_GRAYSCALE)\n        slika = cv2.resize(slika, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)\n        print(str(train_y[index]) + " ==> " + str(img_name))\n    except:\n        print("Ostecena slika " + str(img_name))\n'

In [6]:
import random

random.shuffle(training_data)

In [7]:
X = []
y = []

In [8]:
for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [9]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

Kreiranje CNN-a

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [11]:
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

X = X/255.0

In [12]:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

In [13]:
model.fit(np.array(X), np.array(y), batch_size=9,  epochs=10, validation_split=0.1)

Epoch 1/10
529/529 [==============================] - 194s 366ms/step - loss: 1.5023e-07 - accuracy: 0.2571 - val_loss: 1.5572e-07 - val_accuracy: 0.2476
Epoch 2/10
529/529 [==============================] - 194s 367ms/step - loss: 1.5023e-07 - accuracy: 0.2571 - val_loss: 1.5572e-07 - val_accuracy: 0.2476
Epoch 3/10
144/529 [=======>......................] - ETA: 2:25 - loss: 1.5379e-07 - accuracy: 0.2654

KeyboardInterrupt: 